In [5]:
import pypianoroll 
import pandas as pd
import os
from src.parser import get_raw_segments
from src.classes import ScoreAnnot
from src.annot_types import string_to_instrument

In [3]:
#metadata_file = './dataset/orch_metadata.csv'
midi_path = './dataset/scores_midi'
pianoroll_path = './dataset/scores_pianoroll'
metadata_file = './dataset/new_metadata.csv'

meta_df = pd.read_csv(metadata_file) 
#meta_df = meta_df.drop(12) # hob102 file is damaged
#meta_df = meta_df.drop(range(5)) # mozart32~36 file missing 
meta_df = meta_df.set_index(pd.Index(range(len(meta_df))))
print(meta_df)

'''
# create path to store midi saving path
file_series = meta_df['score']
midi_file_path = []
pianoroll_file_path = []
for path in file_series:
    file_name = path.split('/')[-1].split('.')[0]
    midi_file_path.append( os.path.join(midi_path, file_name+'.mid') )
    pianoroll_file_path.append( os.path.join(pianoroll_path+'/'+file_name+'.npz') ) #Warning: what is file name to store an object

meta_df['midi'] = pd.Series(midi_file_path)
meta_df['pianoroll'] = pd.Series(pianoroll_file_path)
#meta_df.to_csv(new_metadata_file)

print(meta_df) #18 files left'''


    Unnamed: 0.1  Unnamed: 0   composer       piece  \
0              0           0     mozart    mozart38   
1              1           1     mozart    mozart39   
2              2           2     mozart    mozart40   
3              3           3     mozart    mozart41   
4              4           4      haydn    haydn099   
5              5           5      haydn    haydn100   
6              6           6      haydn    haydn101   
7              7           7      haydn    haydn103   
8              8           8      haydn    haydn104   
9              9           9  beethoven  beethoven1   
10            10          10  beethoven  beethoven2   
11            11          11  beethoven  beethoven3   
12            12          12  beethoven  beethoven4   
13            13          13  beethoven  beethoven5   
14            14          14  beethoven  beethoven6   
15            15          15  beethoven  beethoven7   
16            16          16  beethoven  beethoven8   
17        

"\n# create path to store midi saving path\nfile_series = meta_df['score']\nmidi_file_path = []\npianoroll_file_path = []\nfor path in file_series:\n    file_name = path.split('/')[-1].split('.')[0]\n    midi_file_path.append( os.path.join(midi_path, file_name+'.mid') )\n    pianoroll_file_path.append( os.path.join(pianoroll_path+'/'+file_name+'.npz') ) #Warning: what is file name to store an object\n\nmeta_df['midi'] = pd.Series(midi_file_path)\nmeta_df['pianoroll'] = pd.Series(pianoroll_file_path)\n#meta_df.to_csv(new_metadata_file)\n\nprint(meta_df) #18 files left"

In [4]:
for piece in range(meta_df.shape[0]):
    midi_file = meta_df['midi'][piece]
    annotation = meta_df['annotation'][piece]
    score = ScoreAnnot(get_raw_segments(annotation))
    score_inst_list = score.get_inst_list()
    print(score_inst_list)
    
    if os.path.isfile(midi_file):
        
        # load MIDI file into pypianoroll object 
        
        multitrack = pypianoroll.read(midi_file)
        #print('Before pad: ', multitrack.get_max_length())
        multitrack = multitrack.pad(multitrack.resolution*meta_df['n_end_beat'][piece]) 
        #print('After pad: ', multitrack.get_max_length(), ', n_end_beat = ', meta_df['n_end_beat'][piece])
        #print(f'In {midi_file}: ')
        print( len(multitrack.tracks) )
        inst_list = [ track.name for track in multitrack.tracks ]
        print(inst_list)

        # change the name of the tracks
        
        for i in range(len(multitrack.tracks)):
            multitrack.tracks[i].name = score_inst_list[i]
            

        # print track names again
        inst_list = [track.name for track in multitrack.tracks]
        print('After changing track names: ', inst_list)
        
        # save to npz file
        pypianoroll.save(meta_df['pianoroll'][piece], multitrack)
        #print('file_saved: ', meta_df['pianoroll'][piece])
        
    else:
        print(f'ERROR: {midi_file} does not exist!')

['Fl', 'Ob', 'Fg', 'Hrn', 'Trp', 'Timp', 'Vln1', 'Vln2', 'Vla', 'Vc', 'Cb']


/home/yhchu/venv/MIR/texture/lib/python3.10/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


11
['Flute', 'Oboe', 'Bassoon', 'Horn in D', 'C Trumpet', 'Timpani', 'Violins 1', 'Violins 2', 'Violas', 'Violoncellos', 'Contrabasses']
After changing track names:  ['Fl', 'Ob', 'Fg', 'Hrn', 'Trp', 'Timp', 'Vln1', 'Vln2', 'Vla', 'Vc', 'Cb']
['Fl', 'Cl', 'Fg', 'Hrn', 'Trp', 'Timp', 'Vln1', 'Vln2', 'Vla', 'Vc', 'Cb']
11
['Flauto', 'Clarinetti in B', 'Fagotti', 'Corni in Es', 'Trombe in Es', 'Timpani in Es. B.', 'Violino I.', 'Violino II.', 'Viola', 'Violoncello', 'Basso']
After changing track names:  ['Fl', 'Cl', 'Fg', 'Hrn', 'Trp', 'Timp', 'Vln1', 'Vln2', 'Vla', 'Vc', 'Cb']
['Fl', 'Ob', 'Cl', 'Fg', 'HrnB', 'HrnD', 'Vln1', 'Vln2', 'Vla', 'Vc', 'Cb']
11
['Flauto', 'Oboi', 'Clarinetti in B', 'Fagotti', 'Corno in B', 'Corno in G', 'Violino I.', 'Violino II.', 'Viola', 'Violoncello', 'Contrabasso']
After changing track names:  ['Fl', 'Ob', 'Cl', 'Fg', 'HrnB', 'HrnD', 'Vln1', 'Vln2', 'Vla', 'Vc', 'Cb']
['Fl', 'Ob', 'Fg', 'Hrn', 'Trp', 'Timp', 'Vln1', 'Vln2', 'Vla', 'Vc', 'Cb']
11
['Flauto', 

In [4]:
inst_name_dict = {
    'Fl': ['Flute','Flauto','FlÃ¸yte'],
    'Ob': ['Oboe','Oboi','Oboe I','Oboe II','Obo'],
    'Cl': ['Clarinetti in B','SopranoClarinet','SopranoClarinet','Bâ\x99\xad-klarinett','A-klarinett','Bâ\x99\xad Clarinet'],
    'Fg': ['Bassoon','Fagotti','Fagotto I','Fagotto II','Fagott'],
    'Hn': ['Horn in D','Corni in Es','Corno in B','Corno in G','Corno I in C','Corno II in C','Horn','C-horn','D-horn','Eâ\x99\xad-horn',\
           'Horn in Bâ\x99\xad','Horn in Eâ\x99\xad','Horn in F','A-horn','F-horn','Bâ\x99\xad-basshorn'],
    'Trp': ['Trumpet','C Trumpet','Trombe in Es','Clarino I in C','Clarino II in C','C-trompet','D-trompet','Eâ\x99\xad-trompet','Bâ\x99\xad Trumpet',\
           'C Trumpet','F-trompet'],
    'Timp': ['Timpani','Timpani in Es. B.','Timpani in Câ\x80\x93G','Pauker'],
    'Vln1': ['Violins 1','Violino I.','Violin1','Violino I'],
    'Vln2': ['Violins 2','Violino II.','Violin2','Violino II'],
    'Vla': ['Violas','Viola','Bratsjer'],
    'Vc': ['Violoncellos','Violoncello','Celloer'],
    'Cb': ['Contrabasses','Basso','Contrabasso','Contrabass','Kontrabasser']
}

def standard_inst_name(inst_name, inst_name_dict):
    '''Unify the name of instuments'''
    if inst_name=='Violin' or inst_name=='Fioliner':
        print('Map to Vln1 or Vln2?')
    for standard_name in inst_name_dict.keys():
        if inst_name in inst_name_dict[standard_name]:
            return standard_name
    print(f'Cannot find {inst_name}')